### Functions

In [36]:
# read the excel sheet pages and covnert them to the dataframe in pandas
# this excel sheet pages are from 2009 to 2016

def getDfList1():
    '''
    This function will collect cause of fire between 2009 to 2016, remove NAN values,
    and return it as list
    
    Parameters:
    None
    '''
    
    import pandas as pd
    df_16 = [pd.read_excel("2016_Redbook_FINAL.xlsx",sheet_name='Table 13',skiprows=1),
             pd.read_excel("2016_Redbook_FINAL.xlsx",sheet_name='Table 14',skiprows=1)]
    df_15 = [pd.read_excel("2015_Redbook_FINAL.xlsx",sheet_name='Table 11',skiprows=1),
             pd.read_excel("2015_Redbook_FINAL.xlsx",sheet_name='Table 12',skiprows=1),
             pd.read_excel("2015_Redbook_FINAL.xlsx",sheet_name='Table 13',skiprows=1)]
    df_14 = [pd.read_excel("2014_Redbook_Complete.xlsx",sheet_name='Table 11',skiprows=1),
             pd.read_excel("2014_Redbook_Complete.xlsx",sheet_name='Table 12',skiprows=1),
             pd.read_excel("2014_Redbook_Complete.xlsx",sheet_name='Table 13',skiprows=1)]
    df_13 = [pd.read_excel("2013_Redbook_Final.xlsx",sheet_name='Table 12',skiprows=1),
             pd.read_excel("2013_Redbook_Final.xlsx",sheet_name='Table 13',skiprows=1),
             pd.read_excel("2013_Redbook_Final.xlsx",sheet_name='Table 14',skiprows=1)]
    df_12 = [pd.read_excel("2012_Redbook_Complete.xlsx",sheet_name='Table 13',skiprows=1),
             pd.read_excel("2012_Redbook_Complete.xlsx",sheet_name='Table 14',skiprows=1),
             pd.read_excel("2012_Redbook_Complete.xlsx",sheet_name='Table 15',skiprows=1)]
    df_11 = [pd.read_excel("2011_Redbook_Final.xlsx",sheet_name='Table 12',skiprows=1),
             pd.read_excel("2011_Redbook_Final.xlsx",sheet_name='Table 13',skiprows=1),
             pd.read_excel("2011_Redbook_Final.xlsx",sheet_name='Table 14',skiprows=1)]
    df_10 = [pd.read_excel("2010_Redbook_Final_Complete.xlsx",sheet_name='Table 8',skiprows=1),
             pd.read_excel("2010_Redbook_Final_Complete.xlsx",sheet_name='Table 9',skiprows=1),
             pd.read_excel("2010_Redbook_Final_Complete.xlsx",sheet_name='Table 10',skiprows=1)]
    df_09 = [pd.read_excel("2009_Redbook_Complete_Final.xlsx",sheet_name='Table 12',skiprows=2),
             pd.read_excel("2009_Redbook_Complete_Final.xlsx",sheet_name='Table 13',skiprows=2),
             pd.read_excel("2009_Redbook_Complete_Final.xlsx",sheet_name='Table 14',skiprows=2)]
    
    df_list = [df_09, df_10, df_11, df_12, df_13, df_14, df_15, df_16]
    for i in range(len(df_list)):
        for j in range(len(df_list[i])):
            df_list[i][j] = df_list[i][j].dropna(axis=1, thresh=3)
            df_list[i][j] = df_list[i][j].dropna(axis=0, thresh=3)
            df_list[i][j] = df_list[i][j].reset_index(drop=True)
        
    return df_list


def correctName( df_list ):
    '''
    This function will correct the name of the columns to the form o causes
    This function will also correct remove all '\n' in indexes and columns
    
    Parameters:
    df_list -- dataframe list which contains dataframe fire infos of both southern
               region and northern region
    '''
    
    import pandas as pd
    import numpy as np

    for i in range(len(df_list) - 1):
        for j in range(3):
            df_list[i][j] = df_list[i][j].replace('\n',' ', regex=True)
            df_list[i][j] = df_list[i][j].replace('  ',' ', regex=True)
            df_list[i][j] = df_list[i][j].rename(columns={'Powerline': 'Elec. Power'})
            df_list[i][j] = df_list[i][j].rename(columns={'Power Line': 'Elec. Power'})
            df_list[i][j] = df_list[i][j].rename(columns={'Power\nLine': 'Elec. Power'})
            df_list[i][j] = df_list[i][j].rename(columns={'Rail-\nroad': 'Railroad'})
            df_list[i][j] = df_list[i][j].rename(columns={'Rail- road': 'Railroad'})
            df_list[i][j] = df_list[i][j].rename(columns={'Rail\nroad': 'Railroad'})
            df_list[i][j] = df_list[i][j].rename(columns={'Debris\nBurning': 'Debris Burning'})
            df_list[i][j] = df_list[i][j].rename(columns={'Equip.\nUse': 'Equip. Use'})
    
  
    return df_list

In [37]:
def getLocation( cityList ):
    '''
    This function will generate the latitude and longitude of the given city
    list, where these cities are in California. The return value will be 
    dictionary with key as city name.
    
    Parameters:
    cityList -- a list contains all city names
    '''
    
    # import the library
    from geopy.geocoders import Nominatim
    
    # convert cities into dic
    result = []
    
    geolocator = Nominatim()
    
    for city in cityList:
        location = geolocator.geocode( city + ' California' )
        result.append( [float(location.raw['lon']), float(location.raw['lat'])] )
    
    return result


def isDuplicate( city, cityList ):
    '''
    This function will check if the city contains a city in existList (avoid collection)
    
    Parameters:
    city --  city name which will be used to compare to element in cityList
    cityList -- a list contains all the used cities
    '''

    for exist in cityList:
        if exist in city:
            return True
    
    return False


def getCityList( df_list ):
    '''
    This function will use the data frame list provided to collected all cities in the dataframe.
    In each j loop, temp will be reset to empty. This could be used to could each block in excel
    file. Before shift to next sheet, reset temp again.
    
    Parameters:
    df_list -- dataframe list which contains dataframe fire infos of both southern
               region and northern region
    '''
    
    cityListTotal = [] # list contains all seven years city list
    fireListTotal = [] # list contains all seven years fire list
    
    # get city list in 2
    for i in range(len(df_list) - 1):
        cityListCurrent = [] # list in current year
        fireListCurrent = [] # list in current year
        
        # sheet 1
        temp = []
        for j in range(df_list[i][0].shape[0]):
            city = df_list[i][0].loc[j][0]
            
            # if the city exist, skip it
            if isDuplicate( city, temp ):
                temp = [] # clear temp 
                continue
            
            temp.append(city)
            cityListCurrent.append(city)
            fireListCurrent.append(df_list[i][0].loc[j][1])
        
        # sheet 2
        temp = []
        for j in range(df_list[i][1].shape[0]):
            city = df_list[i][1].loc[j][0]
            
            # meet ending, break loop
            if city == 'NORTHERN REGION TOTAL':
                break
            
            # if the city exist, skip it
            if isDuplicate( city, temp ):
                temp = []
                continue
            
            temp.append(city)
            cityListCurrent.append(city)
            fireListCurrent.append(df_list[i][1].loc[j][1])
       
        # sheet 3
        temp = []
        for j in range(df_list[i][2].shape[0]):
            city = df_list[i][2].loc[j][0]
             
            # meet ending, break loop
            if city == 'SOUTHERN REGION TOTAL' or city == 'SOUTHERN  REGION TOTAL' or city == 'STATE TOTAL':
                break
                
            # if the city exist, skip it
            if isDuplicate( city, temp ):
                temp = []
                continue
            
            temp.append(city)
            cityListCurrent.append(city)
            fireListCurrent.append(df_list[i][2].loc[j][1])
               
        cityListTotal.append(cityListCurrent)
        fireListTotal.append(fireListCurrent)
        
    return [cityListTotal, fireListTotal]

In [44]:
def getAllLocation( cityListTotal ):
    '''
    This function will get all location of the givern city list
    
    Parameters:
    cityListTotal -- total list 
    '''
    
    result = []
    
    for i in range( len(cityListTotal) ):
        x = getLocation(cityListTotal[i][:len(cityListTotal[i])/2])
        y = getLocation(cityListTotal[i][len(cityListTotal[i])/2:])
        result.append(x + y)
        
    return result

In [45]:
def write(k):
    '''
    This function will write datas into csv file
    '''
    
    fname = 'Data_%d.csv'%(k+2009)
    f = open(fname,'w')
    for i in range(len(allLocation[k])):
        f.write('%s,%f,%f,%f\n' % (cityListTotal[k][i],allLocation[k][i][0],allLocation[k][i][1],float(fireListTotal[k][i])))
    f.close()

    return

### Execute functions above

In [ ]:
df_list = correctName( getDfList1() )
[cityListTotal, fireListTotal] = getCityList( df_list )
allLocation = getAllLocation( cityListTotal )

for j in range(7):
    write(j)